# Build zRank

In [ ]:
import matplotlib.pyplot as plt
import numexpr
import numpy as np
import pandas as pd
from pathlib import Path

%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 5]

In [ ]:
import sys
sys.path.append("ZH")

from higgsstrahlung.from_root import RootfileHandler
import higgsstrahlung.cuts as cuts


rf = "2020-10-08-235437"
rf = Path.home() / "data/ZH/DataFrames" / rf

file_handler = RootfileHandler(tupleName="e1Tree", root_folder=rf)
e1_events = file_handler._df
e1_signal = e1_events[e1_events.process == "e1e1h"]
# meta = pd.read_pickle(file_handler._r2p.df_folder / "meta.pkl")
# preselection = "(" + ") & (".join(cuts.preselections["e1Tree"]) + ")"

file_handler = RootfileHandler(tupleName="e2Tree", root_folder=rf)
e2_events = file_handler._df
e2_signal = e2_events[e2_events.process == "e2e2h"]

file_handler = RootfileHandler(tupleName="nuTree", root_folder=rf)
nu_events = file_handler._df
nu_signal = nu_events[nu_events.process == "nnh"]

## Build a ranking

In [ ]:
from z_rank import createZRanking

e1_cl = createZRanking(e1_signal, e1_events, 1_000, print_every=100, save_as="e1_eLpR")

In [ ]:
e1_cl

In [ ]:
e2_cl = createZRanking(e2_signal, e2_events, 1_000, print_every=100, save_as="e2_eLpR")

In [ ]:
#%%capture out  
# Do not show the figure. 

def addPreviouslyProposedCutLines(axs):
    kw = dict(color="orange", ls=":", label="proposed by-hand cut")
    axs[0].axvline(88, **kw)
    axs[0].axvline(94, **kw)
    axs[1].axvline(124, **kw)
    axs[1].axvline(127, **kw)
    axs[2].axvline(.99, **kw)
    axs[3].axvline(.93, **kw)

cl = e1_cl

x = cl.index / 10
#fig, axs = plt.subplots(ncols=5, figsize=(6, 4), sharey=True)
#fig, axs = plt.subplots(ncols=5, figsize=(10, 25), sharex=True)
fig, ax_dict = plt.subplot_mosaic("0123\n0123\n0123\n0123\n4444\n4444\n4444", figsize=(6, 5))
axs = [ax_dict[k] for k in sorted(ax_dict)]

kw_fill = dict(alpha=.9, label="eff*pur ranked cut")
axs[0].fill_betweenx(x, cl["mZ >= "], cl["mZ <= "], **kw_fill)
axs[1].fill_betweenx(x, cl["mRecoil <= "], cl["mRecoil >= "], **kw_fill)
axs[2].fill_betweenx(x, 0, cl["abs(cosTMiss) <= "], **kw_fill)
axs[3].fill_betweenx(x, 0, cl["abs(cosTZ) <= "], **kw_fill)
axs[0].get_shared_y_axes().join(*axs[:-1])
for ax in axs[1:-1]:
    ax.set_yticklabels([])
axs[0].set_ylim((0, 100))

axs[0].set_xlabel("mZ")
axs[1].set_xlabel("mRecoil")
axs[2].set_xlabel("abs(cosTMiss)")
axs[3].set_xlabel("abs(cosTZ)")
axs[0].set_ylabel("1 - signal efficiency [%]")
axs[-1].set_xlabel("1 - signal efficiency [%]")

addPreviouslyProposedCutLines(axs)

axs[-1].plot(x, cl.pur, label="pur")
axs[-1].plot(x, cl.eff, label="eff")
axs[-1].plot(x, cl.eff * cl.pur, label="eff*pur")
for i in range(len(axs) - 1):
    axs[i].axhline((cl.eff * cl.pur).argmax() / 10, color="gray", ls="--", label="max(eff*pur)")
axs[-1].axvline((cl.eff * cl.pur).argmax() / 10, color="gray", ls="--", label="max(eff*pur)")

axs[-1].set_xlim((0, x.max()))
axs[-2].legend()
axs[-2].set_zorder(2)
axs[-1].legend()

fig.subplots_adjust(hspace=3)
fig.savefig("fig/z_ranking.png", facecolor=None, dpi=300)

In [ ]:
print((e1_cl.eff * e1_cl.pur).max())
e1_cl.iloc[(e1_cl.eff * e1_cl.pur).argmax()]

In [ ]:
print((e2_cl.eff * e2_cl.pur).max())
e2_cl.iloc[(e2_cl.eff * e2_cl.pur).argmax()]

## Add the zRank variable

In [ ]:
from z_rank import getZRank

e1_events["zRank"] = getZRank(e1_events, ranking_table="e1_eLpR")
e2_events["zRank"] = getZRank(e2_events, ranking_table="e2_eLpR")

In [ ]:
e1_signal = e1_events[e1_events.process == "e1e1h"]

e1_events.zRank.plot.hist(bins=200, weights=e1_events.weight, label="all")
e1_signal.zRank.plot.hist(bins=200, weights=e1_signal.weight, label="e1e1h")
plt.yscale("log")
plt.xlabel("Z$_e$")
plt.legend();

In [ ]:
e2_signal = e2_events[e2_events.process == "e2e2h"]

e2_events.zRank.plot.hist(bins=200, weights=e2_events.weight, label="all")
e2_signal.zRank.plot.hist(bins=200, weights=e2_signal.weight, label="e2e2h")
plt.yscale("log")
plt.xlabel("Z$_e$")
plt.legend();

## Add the $\nu\nu$-BDT$_H$

In [ ]:
df_path = Path("data")
if (df_path / "nu_events.pkl").exists():
    nu_events = pd.read_pickle(df_path / "nu_events.pkl")
    e1_events = pd.read_pickle(df_path / "e1_events.pkl")
    e2_events = pd.read_pickle(df_path / "e2_events.pkl")
else:
    from higgs_only_model import getXGBModel

    model, training_columns = getXGBModel()
    nu_events["hBDT"] = model.predict_proba(nu_events[training_columns])[:,1] 
    e1_events["hBDT"] = model.predict_proba(e1_events[training_columns])[:,1] 
    e2_events["hBDT"] = model.predict_proba(e2_events[training_columns])[:,1] 

    nu_events.to_pickle(df_path / "nu_events.pkl")
    e1_events.to_pickle(df_path / "e1_events.pkl")
    e2_events.to_pickle(df_path / "e2_events.pkl")

In [ ]:
e1_signal = e1_events[e1_events.process == "e1e1h"]

e1_events.hBDT.plot.hist(bins=200, weights=e1_events.weight, label="all")
e1_signal.hBDT.plot.hist(bins=200, weights=e1_signal.weight, label="e1e1h")
plt.yscale("log")
plt.xlabel("BDT$_H$")
plt.legend();

In [ ]:
e2_signal = e2_events[e2_events.process == "e2e2h"]

e2_events.hBDT.plot.hist(bins=200, weights=e2_events.weight, label="all")
e2_signal.hBDT.plot.hist(bins=200, weights=e2_signal.weight, label="e2e2h")
plt.yscale("log")
plt.xlabel("BDT$_H$")
plt.legend();

In [ ]:
nu_signal = nu_events[nu_events.process == "nnh"]

fig, ax = plt.subplots(figsize=(4, 3))
nu_events.hBDT.plot.hist(bins=200, weights=nu_events.weight, label="all SM")
nu_signal.hBDT.plot.hist(ax=ax, bins=200, weights=nu_signal.weight, label="ννH")
ax.set_yscale("log")
ax.set_xlabel("BDT$_H$")

ax.axvline(.18, color="black", linewidth=3)
y_mean = pow(10, np.log10(ax.get_ylim()[1]) - np.log10(ax.get_ylim()[0]))
y_mean = pow(10, 0.5 * (np.log10(ax.get_ylim()[1]) + np.log10(ax.get_ylim()[0])))
ax.arrow(  .18, y_mean, .075, 0, length_includes_head=True,
          head_width=y_mean/5, head_length=0.025, linewidth=2, color="black",
          label="Selection efficiency from eeH, μμH")
ax.set_title("Higgs production channel agnostic BDT for ννH selection")
ax.set_title("Higgs production agnostic BDT for ννH")
ax.legend()
fig.tight_layout()
fig.savefig("fig/new_ext_nnH_BDT_production_agnostic.png", facecolor=None, dpi=300)


In [ ]:
nu_events

## Appendix:
A H ranking for the Higgs part of ZH, Z->$\nu\nu$ events.
(Was produced earlier along the lines of the Z ranking. Here only loaded.)

from z_rank import getZRank

nu_events["hRank"] = getZRank(nu_events, ranking_table="nu_eLpR")
nu_signal = nu_events[nu_events.process == "nnh"]
nu_events.hRank.plot.hist(bins=200, weights=nu_events.weight, label="all")
nu_signal.hRank.plot.hist(bins=200, weights=nu_signal.weight, label="nnh")
plt.yscale("log")
plt.xlabel("BDT$_H$")
plt.legend();